In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,569 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,961 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,353 kB]
Get:13 https://developer.download.nvidia.com/compute

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df= spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [5]:
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
from pyspark.sql.functions import year

# Show the year for the date column
home_sales_df.select(year(home_sales_df["date"])).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [7]:
home_sales_df = home_sales_df.withColumn("year", year(home_sales_df['date']))
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [8]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('sales')

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT year, bedrooms, ROUND(AVG(price), 2) AS average_price FROM sales WHERE bedrooms==4 GROUP BY year, bedrooms""" ).show()


+----+--------+-------------+
|year|bedrooms|average_price|
+----+--------+-------------+
|2020|       4|    298353.78|
|2019|       4|     300263.7|
|2022|       4|    296363.88|
|2021|       4|    301819.44|
+----+--------+-------------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT year, bedrooms, bathrooms, ROUND(AVG(price), 2) AS average_price FROM sales WHERE bedrooms==3 and bathrooms=3  GROUP BY year, bedrooms, bathrooms""" ).show()



+----+--------+---------+-------------+
|year|bedrooms|bathrooms|average_price|
+----+--------+---------+-------------+
|2021|       3|        3|    294211.46|
|2020|       3|        3|    294204.16|
|2022|       3|        3|    292725.69|
|2019|       3|        3|    287287.82|
+----+--------+---------+-------------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("""SELECT year, bedrooms, bathrooms, floors, sqft_living, ROUND(AVG(price), 2) AS average_price FROM sales WHERE bedrooms==3 and bathrooms=3 and floors==2 and sqft_living > 2000 GROUP BY year, bedrooms, bathrooms, floors, sqft_living""" ).show()


+----+--------+---------+------+-----------+-------------+
|year|bedrooms|bathrooms|floors|sqft_living|average_price|
+----+--------+---------+------+-----------+-------------+
|2020|       3|        3|     2|       2368|     198931.0|
|2021|       3|        3|     2|       2147|     369045.0|
|2022|       3|        3|     2|       2224|     386247.0|
|2021|       3|        3|     2|       2450|     214532.0|
|2020|       3|        3|     2|       2267|     316568.0|
|2019|       3|        3|     2|       2467|     198456.0|
|2021|       3|        3|     2|       2211|     261228.0|
|2022|       3|        3|     2|       2294|     261886.0|
|2021|       3|        3|     2|       3430|     697736.0|
|2021|       3|        3|     2|       2365|     177896.0|
|2020|       3|        3|     2|       2280|     248526.0|
|2020|       3|        3|     2|       2314|     156188.0|
|2020|       3|        3|     2|       2427|     130176.0|
|2019|       3|        3|     2|       2208|     257037.

In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view, ROUND(AVG(price), 2) AS average_price FROM sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""" ).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.0770199298858643 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()


spark.sql("""SELECT view, ROUND(AVG(price), 2) AS average_price FROM sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""" ).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.7203056812286377 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("sales")

In [23]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('sales')

In [24]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_sales_p')

In [25]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""SELECT view, ROUND(AVG(price), 2) AS average_price FROM sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""" ).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.4708714485168457 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [27]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("sales")

False